In [52]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc
import requests
import os
import time

In [53]:
# 다운로드 경로 설정
download_path = "/downloads1"

chrome_options = uc.ChromeOptions()
chrome_options.add_experimental_option("prefs", {
  "profile.default_content_setting_values.popups": 1,  # 0: 모든 팝업 차단, 1: 모든 팝업 허용
  "profile.default_content_setting_values.notifications": 1,  # 1: 허용, 2: 차단
  "download.default_directory": download_path,  # 다운로드 경로
  "download.prompt_for_download": False,  # 다운로드시 확인 창 띄우지 않음
  "download.directory_upgrade": True,  # 다운로드 경로 설정 활성화
  "safebrowsing.enabled": True  # 안전 브라우징 활성화
})

# 드라이버 설정
driver = uc.Chrome(options=chrome_options)

In [54]:
# 페이지 접근
driver.get('https://play.google.com/books/reader?id=6ybeEAAAQBAJ&hl=ko')

In [70]:
# iframe 대기 및 포커스 이동
iframe = WebDriverWait(driver, 1).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe.-gb-display"))
)
driver.switch_to.frame(iframe)

# 필요한 요소 대기 및 찾기
wrapper = WebDriverWait(driver, 1).until(
    EC.presence_of_element_located((By.CLASS_NAME, "cdk-virtual-scroll-content-wrapper"))
)

In [71]:
def func(wrapper, list):
    ol_element = wrapper.find_element(By.TAG_NAME, "ol")
    li_elements = ol_element.find_elements(By.TAG_NAME, "li")

    for li in li_elements:
        render = li.find_element(By.TAG_NAME, "reader-rendered-page")
        img = render.find_element(By.TAG_NAME, "img")
        #src_value = img.get_attribute('src')
        src_value = img.get_attribute('real-src')
        print(src_value)
        if src_value not in list:
            list.append(src_value)
            last_index = len(list) - 1
            download_selenium(src_value, last_index)
            return

# # iframe 내에서 스크롤
# def scroll(wrapper):
#     ol_element = wrapper.find_element(By.TAG_NAME, "ol")
#     li_elements = ol_element.find_elements(By.TAG_NAME, "li")
#     driver.execute_script("arguments[0].scrollIntoView();", li_elements[-1])

def download_selenium(url, temp):
    driver.execute_script("var link = document.createElement('a'); link.href = arguments[0]; link.download = ''; document.body.appendChild(link); link.click();", url)
    """
    original_tab = driver.current_window_handle

    # 새 탭 열기
    driver.execute_script("window.open('');")
    new_tab = driver.window_handles[-1]
    driver.switch_to.window(new_tab)
    driver.get(url)

    new_tab_img = WebDriverWait(driver, 1).until(
        EC.presence_of_element_located((By.TAG_NAME, "img"))
    )

    driver.execute_script("window.open('{}');".format(url))
    time.sleep(5)
    """

In [72]:
list = []
func(wrapper, list)

https://play.google.com/books/publisher/content/reader?id=6ybeEAAAQBAJ&hl=ko&pg=PP1&img=1&zoom=3&sig=ACfU3U18CswFVEPESa1WkhaIK9JvTxV12A&w=663&h=856&edge=stretch


In [69]:
driver.switch_to.window(driver.window_handles[0])